In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, QuatE, RotatE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive_sampled.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     validation_path = VALIDATE_PATH,
                      testing_path = TEST_PATH,
                      eager = True
                     )


In [5]:
model_name = 'rotatE_transductive_sampling'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [8]:
model = RotatE(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 6665200
Space occupied: 26660800 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory rotatE_transductive_sampling created successfully!


In [10]:
learning_rate = 5e-2 #1e-3, and 1e-4 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
#         callbacks="evaluation",
#         callback_kwargs=dict(
#             evaluator=valid_evaluator,
#             evaluation_triples=dataset.validation.mapped_triples,
#             prefix="validation",
#             frequency=1,
#             additional_filter_triples=dataset.training.mapped_triples,
#         ),
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 3.5007872581481934
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0003022818584958561
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.00012471556494454098
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.00021349871172019854
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.00030228172545321286
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.00012471548689063638
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.00021349861344788224
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0003022816588789847
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00012471540346241963
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00021349853117070218
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validati

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 2.9338836669921875
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0003236491004940404
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.00010978882723834415
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0002167189638661923
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.00032364900107495487
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.00010978877253364772
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.00021671886497642845
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.00032364888812735085
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00010978871438697103
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00021671880125716088
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validati

...

Step: 186
Metric: loss = 0.03358152508735657
Step: 186
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.15927350582116412
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.3534465260049838
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.256360015913074
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.15927350521087646
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.3534465432167053
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.2563600242137909
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.15927350581942737
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.3534465260040376
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.25636001591173246
Metric: validation.head.optimistic.hits_at_1 = 0.0774862721171446
Metric: validation.tail.optimistic.hits_at_1 = 0.24176327028676023
Metric: validation.both.optimistic.hits_at_1 = 0.1596247712019

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 187
Metric: loss = 0.033987775444984436
Step: 187
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.15947322648381904
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.35251127803932286
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.2559922522615709
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.1594732254743576
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.35251128673553467
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.25599223375320435
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.1594732219777224
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.35251127420600675
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.25599224809186455
Metric: validation.head.optimistic.hits_at_1 = 0.07763880414887127
Metric: validation.tail.optimistic.hits_at_1 = 0.23993288590604026
Metric: validation.both.optimistic.hits_at_1 = 0.15878584

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 188
Metric: loss = 0.03407420963048935
Step: 188
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.15949104584200113
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.3530919843759215
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.2562915151089613
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.15949104726314545
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.3530919849872589
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.2562915086746216
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.15949104572020448
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.35309198395577746
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.25629151483799095
Metric: validation.head.optimistic.hits_at_1 = 0.07733374008541793
Metric: validation.tail.optimistic.hits_at_1 = 0.24115314215985356
Metric: validation.both.optimistic.hits_at_1 = 0.1592434411

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    3.500787
1    2.933884
2    2.363729
3    1.814819
4    1.318954
..        ...
183  0.034680
184  0.035042
185  0.033582
186  0.033988
187  0.034074

[188 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/26.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.907786   0.907786     0.907786
hits_at_1                     0.877516   0.877516     0.877516
hits_at_3                     0.928110   0.928110     0.928110
hits_at_5                     0.945025   0.945025     0.945025
hits_at_10                    0.965754   0.965754     0.965754
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.929812   0.929811     0.929810
hits_at_1                     0.906249   0.906249     0.906249
hits_at_3                     0.946158   0.946158     0.946158
hits_at_5                     0.958467   0.958467     0.958467
hits_at_10                    0.970361   0.970361     0.970361
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.918799   0.918799     0.918798
hits_at_1                     0.891882   0.891882     0.891882
hits_at_3                     0.937134  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/6.56k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.157427   0.157427     0.157427
hits_at_1                     0.078707   0.078707     0.078707
hits_at_3                     0.168853   0.168853     0.168853
hits_at_5                     0.225290   0.225290     0.225290
hits_at_10                    0.317572   0.317572     0.317572
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.350587   0.350587     0.350587
hits_at_1                     0.236272   0.236272     0.236272
hits_at_3                     0.417328   0.417328     0.417328
hits_at_5                     0.492678   0.492678     0.492678
hits_at_10                    0.574436   0.574436     0.574436
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.254007   0.254007     0.254007
hits_at_1                     0.157489   0.157489     0.157489
hits_at_3                     0.293090  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/8.34k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.270320   0.270320     0.270320
hits_at_1                     0.187822   0.187822     0.187822
hits_at_3                     0.293300   0.293300     0.293300
hits_at_5                     0.348556   0.348556     0.348556
hits_at_10                    0.442527   0.442527     0.442527
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.414074   0.414071     0.414070
hits_at_1                     0.322905   0.322905     0.322905
hits_at_3                     0.466619   0.466619     0.466619
hits_at_5                     0.517560   0.517440     0.517440
hits_at_10                    0.574973   0.574973     0.574973
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.342197   0.342196     0.342195
hits_at_1                     0.255364   0.255364     0.255364
hits_at_3                     0.379959  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   

                                                value  
0                                              cuda:0  
1                                            266608

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)